Wikipedia Retrievers

In [10]:
from langchain_community.retrievers import WikipediaRetriever

In [11]:
# Initialize the retriever
retriever = WikipediaRetriever(top_k_results=2,lang='en')

In [14]:
query = "India Pakistan relations"

# get relevant wikipeadia documents
docs= retriever.invoke(query) # retriever is a runnabel

In [15]:
docs

[Document(metadata={'title': 'India–Pakistan relations', 'summary': 'India and Pakistan have a complex and largely hostile relationship that is rooted in a multitude of historical and political events, most notably the partition of British India in August 1947.\nTwo years after World War II, the United Kingdom formally dissolved British India, dividing it into two new sovereign nations: the Union of India and Pakistan. The partitioning of the former British colony resulted in the displacement of up to 15 million people, with the death toll estimated to have reached between several hundred thousand and one million people as Hindus and Muslims migrated in opposite directions across the Radcliffe Line to reach India and Pakistan, respectively. In 1950, India emerged as a secular republic with a Hindu-majority population. Shortly afterwards, in 1956, Pakistan emerged as an Islamic republic with a Muslim-majority population.\nWhile the two South Asian countries established full diplomatic t

In [17]:
# print retrieved content
for i,doc in enumerate(docs):
    print(f"\n----Result {i+1}")
    print(f"content: \n{doc.page_content}....")


----Result 1
content: 
India and Pakistan have a complex and largely hostile relationship that is rooted in a multitude of historical and political events, most notably the partition of British India in August 1947.
Two years after World War II, the United Kingdom formally dissolved British India, dividing it into two new sovereign nations: the Union of India and Pakistan. The partitioning of the former British colony resulted in the displacement of up to 15 million people, with the death toll estimated to have reached between several hundred thousand and one million people as Hindus and Muslims migrated in opposite directions across the Radcliffe Line to reach India and Pakistan, respectively. In 1950, India emerged as a secular republic with a Hindu-majority population. Shortly afterwards, in 1956, Pakistan emerged as an Islamic republic with a Muslim-majority population.
While the two South Asian countries established full diplomatic ties shortly after their formal independence, th

Vector Store Retrievers

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

d:\langchain\Langchain models\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [3]:
# Step 2: Initialize embedding model
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_collection"
)

In [4]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [5]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [6]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


MMR

In [7]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [8]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

In [9]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [10]:
query = "What is langchain?"
results = retriever.invoke(query)

In [11]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
MMR helps you get diverse results when doing similarity search.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


Multiquery Retriever

In [25]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

In [26]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [27]:
# Initialize OpenAI embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embeddings)

In [28]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [29]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash')
)

In [30]:
# Query
query = "How to improve energy levels and maintain balance?"

In [31]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [32]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and 

Contextual Compression Rtrievers

In [33]:
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [34]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [36]:
# Initialize OpenAI embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [37]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [38]:
# Set up the compressor using an LLM
llm = ChatGoogleGenerativeAI(model='models/gemini-2.5-flash')
compressor = LLMChainExtractor.from_llm(llm)

In [39]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [40]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [41]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis does not occur in animal cells.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
